<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Análisis de sentimiento
`Fecha de entrega: Noviembre 21, 2020 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Hacer análisis de sentimiento de los datos del anterior taller
- Columna con el puntaje numérico del sentimiento
- Pueden usar cualquiera de las librerias vistas en clase

In [1]:
import pandas as pd 
import tweepy

In [2]:
keys = pd.read_csv('D:/Users/User/Imágenes/Especializacion Analitica Estrategica de Datos/Semestre II/Electiva NLP/clase_12/keys_demo.txt', header=None)
keys = dict(zip(keys[0],keys[1]))
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)
tweets = tweepy.Cursor(api.search,
                       q=['partido -RT'],
                       lang='es',
                       geocode=f'4.649178,-74.062827,45mi',
                       tweet_mode='extended',
                       include_rts=False).items(100)
t=[]
for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append("Bogotá")
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    
    t.append(temp)
    data = pd.DataFrame(t)
data.columns = ['tweet', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweet,date,city,user_location,id,link
0,@Edwin_Pizza @ElDelMedallo Pero si es al contr...,2020-11-21 04:46:56,Bogotá,Bogota,1330009777909411841,https://twitter.com/i/web/status/1330009777909...
1,"En la convención nacional conservadora, que se...",2020-11-21 04:45:02,Bogotá,"Bogotá, Colombia",1330009302493433857,https://twitter.com/i/web/status/1330009302493...
2,@reinaldobernalv @petrogustavo 1) Si se ha tom...,2020-11-21 04:39:29,Bogotá,Bogota,1330007903135207424,https://twitter.com/i/web/status/1330007903135...
3,@NoticiasRCN Como puede lograr un pueblo encon...,2020-11-21 04:38:45,Bogotá,Bogotá - Colombia,1330007720343252993,https://twitter.com/i/web/status/1330007720343...
4,@DanielSamperO @CathyJuvinao Siempre he recono...,2020-11-21 04:35:22,Bogotá,Bogota,1330006869625237505,https://twitter.com/i/web/status/1330006869625...


In [3]:
import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

data[['clean', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')

data.head()

,tweet,date,city,user_location,id,link,clean,links,hashtags,mentions
0,@Edwin_Pizza @ElDelMedallo Pero si es al contr...,2020-11-21 04:46:56,Bogotá,Bogota,1330009777909411841,https://twitter.com/i/web/status/1330009777909...,"Pero si es al contrario, con eso le dan la r...",[],[],"[@Edwin_Pizza, @ElDelMedallo]"
1,"En la convención nacional conservadora, que se...",2020-11-21 04:45:02,Bogotá,"Bogotá, Colombia",1330009302493433857,https://twitter.com/i/web/status/1330009302493...,"En la convención nacional conservadora, que se...",[https://t.co/Lz1R4DCD5I],[],[]
2,@reinaldobernalv @petrogustavo 1) Si se ha tom...,2020-11-21 04:39:29,Bogotá,Bogota,1330007903135207424,https://twitter.com/i/web/status/1330007903135...,1) Si se ha tomado una posicion que no es la...,[],[],"[@reinaldobernalv, @petrogustavo]"
3,@NoticiasRCN Como puede lograr un pueblo encon...,2020-11-21 04:38:45,Bogotá,Bogotá - Colombia,1330007720343252993,https://twitter.com/i/web/status/1330007720343...,"Como puede lograr un pueblo encontrar ""PAZ"" c...",[],[],[@NoticiasRCN]
4,@DanielSamperO @CathyJuvinao Siempre he recono...,2020-11-21 04:35:22,Bogotá,Bogota,1330006869625237505,https://twitter.com/i/web/status/1330006869625...,"Siempre he reconocido ideas buenas, progresi...",[],[],"[@DanielSamperO, @CathyJuvinao]"


In [4]:
from textblob import TextBlob 
from googletrans import Translator
#TextBlob(ejemplo5).sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

vader = SentimentIntensityAnalyzer()
#vader.polarity_scores(ejemplo5)['compound']

In [5]:
translator = Translator()
data['ing'] = data['clean'].apply(lambda valor: translator.translate(valor, src='es', dest='en'))
data.head()

,tweet,date,city,user_location,id,link,clean,links,hashtags,mentions,ing
0,@Edwin_Pizza @ElDelMedallo Pero si es al contr...,2020-11-21 04:46:56,Bogotá,Bogota,1330009777909411841,https://twitter.com/i/web/status/1330009777909...,"Pero si es al contrario, con eso le dan la r...",[],[],"[@Edwin_Pizza, @ElDelMedallo]","Translated(src=es, dest=en, text=But if it is ..."
1,"En la convención nacional conservadora, que se...",2020-11-21 04:45:02,Bogotá,"Bogotá, Colombia",1330009302493433857,https://twitter.com/i/web/status/1330009302493...,"En la convención nacional conservadora, que se...",[https://t.co/Lz1R4DCD5I],[],[],"Translated(src=es, dest=en, text=In the conser..."
2,@reinaldobernalv @petrogustavo 1) Si se ha tom...,2020-11-21 04:39:29,Bogotá,Bogota,1330007903135207424,https://twitter.com/i/web/status/1330007903135...,1) Si se ha tomado una posicion que no es la...,[],[],"[@reinaldobernalv, @petrogustavo]","Translated(src=es, dest=en, text=1) If you hav..."
3,@NoticiasRCN Como puede lograr un pueblo encon...,2020-11-21 04:38:45,Bogotá,Bogotá - Colombia,1330007720343252993,https://twitter.com/i/web/status/1330007720343...,"Como puede lograr un pueblo encontrar ""PAZ"" c...",[],[],[@NoticiasRCN],"Translated(src=es, dest=en, text=How can a peo..."
4,@DanielSamperO @CathyJuvinao Siempre he recono...,2020-11-21 04:35:22,Bogotá,Bogota,1330006869625237505,https://twitter.com/i/web/status/1330006869625...,"Siempre he reconocido ideas buenas, progresi...",[],[],"[@DanielSamperO, @CathyJuvinao]","Translated(src=es, dest=en, text=I have always..."


In [6]:
data['ingles'] = data['ing'].apply(lambda valor: valor.text)
data.head()

,tweet,date,city,user_location,id,link,clean,links,hashtags,mentions,ing,ingles
0,@Edwin_Pizza @ElDelMedallo Pero si es al contr...,2020-11-21 04:46:56,Bogotá,Bogota,1330009777909411841,https://twitter.com/i/web/status/1330009777909...,"Pero si es al contrario, con eso le dan la r...",[],[],"[@Edwin_Pizza, @ElDelMedallo]","Translated(src=es, dest=en, text=But if it is ...","But if it is the opposite, with that they agre..."
1,"En la convención nacional conservadora, que se...",2020-11-21 04:45:02,Bogotá,"Bogotá, Colombia",1330009302493433857,https://twitter.com/i/web/status/1330009302493...,"En la convención nacional conservadora, que se...",[https://t.co/Lz1R4DCD5I],[],[],"Translated(src=es, dest=en, text=In the conser...","In the conservative national convention, which..."
2,@reinaldobernalv @petrogustavo 1) Si se ha tom...,2020-11-21 04:39:29,Bogotá,Bogota,1330007903135207424,https://twitter.com/i/web/status/1330007903135...,1) Si se ha tomado una posicion que no es la...,[],[],"[@reinaldobernalv, @petrogustavo]","Translated(src=es, dest=en, text=1) If you hav...",1) If you have taken a position that is not th...
3,@NoticiasRCN Como puede lograr un pueblo encon...,2020-11-21 04:38:45,Bogotá,Bogotá - Colombia,1330007720343252993,https://twitter.com/i/web/status/1330007720343...,"Como puede lograr un pueblo encontrar ""PAZ"" c...",[],[],[@NoticiasRCN],"Translated(src=es, dest=en, text=How can a peo...","How can a people find ""PEACE"" when there is no..."
4,@DanielSamperO @CathyJuvinao Siempre he recono...,2020-11-21 04:35:22,Bogotá,Bogota,1330006869625237505,https://twitter.com/i/web/status/1330006869625...,"Siempre he reconocido ideas buenas, progresi...",[],[],"[@DanielSamperO, @CathyJuvinao]","Translated(src=es, dest=en, text=I have always...","I have always recognized good, progressive ide..."


In [7]:
data['sentimiento'] = data['ingles'].apply(lambda valor: vader.polarity_scores(valor)['compound'])
data.head()

,tweet,date,city,user_location,id,link,clean,links,hashtags,mentions,ing,ingles,sentimiento
0,@Edwin_Pizza @ElDelMedallo Pero si es al contr...,2020-11-21 04:46:56,Bogotá,Bogota,1330009777909411841,https://twitter.com/i/web/status/1330009777909...,"Pero si es al contrario, con eso le dan la r...",[],[],"[@Edwin_Pizza, @ElDelMedallo]","Translated(src=es, dest=en, text=But if it is ...","But if it is the opposite, with that they agre...",-0.0387
1,"En la convención nacional conservadora, que se...",2020-11-21 04:45:02,Bogotá,"Bogotá, Colombia",1330009302493433857,https://twitter.com/i/web/status/1330009302493...,"En la convención nacional conservadora, que se...",[https://t.co/Lz1R4DCD5I],[],[],"Translated(src=es, dest=en, text=In the conser...","In the conservative national convention, which...",0.4215
2,@reinaldobernalv @petrogustavo 1) Si se ha tom...,2020-11-21 04:39:29,Bogotá,Bogota,1330007903135207424,https://twitter.com/i/web/status/1330007903135...,1) Si se ha tomado una posicion que no es la...,[],[],"[@reinaldobernalv, @petrogustavo]","Translated(src=es, dest=en, text=1) If you hav...",1) If you have taken a position that is not th...,-0.0258
3,@NoticiasRCN Como puede lograr un pueblo encon...,2020-11-21 04:38:45,Bogotá,Bogotá - Colombia,1330007720343252993,https://twitter.com/i/web/status/1330007720343...,"Como puede lograr un pueblo encontrar ""PAZ"" c...",[],[],[@NoticiasRCN],"Translated(src=es, dest=en, text=How can a peo...","How can a people find ""PEACE"" when there is no...",0.7672
4,@DanielSamperO @CathyJuvinao Siempre he recono...,2020-11-21 04:35:22,Bogotá,Bogota,1330006869625237505,https://twitter.com/i/web/status/1330006869625...,"Siempre he reconocido ideas buenas, progresi...",[],[],"[@DanielSamperO, @CathyJuvinao]","Translated(src=es, dest=en, text=I have always...","I have always recognized good, progressive ide...",-0.0583


In [10]:
data_def=data[['clean', 'sentimiento']].sort_values('sentimiento')
data_def

,clean,sentimiento
54,son Yam hijos de puta que sacan eso hasta hor...,-0.9062
53,Cómo me vas a avisar un día antes del clásico...,-0.8724
10,"Se juntaron el hambre con la necesidad, o mej...",-0.8360
67,Hps los dos no fueron capaz de aceptar que lo...,-0.8118
49,"Pero que lo pongan de una en el reglamento,...",-0.7720
...,...,...
55,Se cambian de partido como de ropa 😂😂😂,0.9169
41,Una buena opción para presidente y...,0.9287
83,Santafe es muy fuerte pero el actual campeón...,0.9437
31,"Se llama 21 personalidades, algo así como fr...",0.9460


###  `[10 pts]` Punto 2: ¿Cuál es el tuit más positivo y más negativo?

In [11]:
mas_positivo = data_def.sort_values('sentimiento',ascending=False)
mas_positivo.head(1)

,clean,sentimiento
28,Jajajaja se fue hasta mi perfil? Jajajaja m...,0.9867


In [12]:
mas_negativo = data_def.sort_values('sentimiento',ascending=True)
mas_negativo.head(1)

,clean,sentimiento
54,son Yam hijos de puta que sacan eso hasta hor...,-0.9062


###  `[15 pts]` Punto 3: ¿Cuántos tuits positivos/negativos hay en sus datos?

In [21]:
print("No. de Tuist positivos",len(data_def[data_def.sentimiento > 0]))


No. de Tuist positivos 50


In [22]:
print("No. de Tuist negativos",len(data_def[data_def.sentimiento <= 0]))

No. de Tuist negativos 50


###  `[BONUS 5pts]` : Diagrama de barras de la pregunta 3

###  `[BONUS 7pts]` : Nubes de palabras por sentimiento